In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from utils import *

In [ ]:
@jit(nopython=True, parallel=True)
def nantrend_custom(var_values):
    """
    计算给定数组中正数和负数的数量，忽略 NaN 值。
    
    参数:
    - var_values (array-like): 输入的数值数组。
    
    返回:
    - tuple: 包含两个元素的元组，第一个元素为正数的数量，第二个元素为负数的数量。
    """
    pos_count = 0
    neg_count = 0
    for val in var_values:
        if not np.isnan(val):
            if val > 0:
                pos_count += 1
            elif val < 0:
                neg_count += 1
    return (pos_count, neg_count)

In [ ]:
@jit(nopython=True, parallel=True)
def get_seconds_var_mean_st_ed(var_values, backward_idx_st, backward_idx_ed):
    var_mean_values = np.full(len(var_values), np.nan)
    for i in prange(len(var_values)):
        start_idx = backward_idx_st[i]
        end_idx = backward_idx_ed[i]
        if i > start_idx and end_idx > start_idx:
            var_subset = var_values[start_idx:end_idx+1]
            var_mean_values[i] = nanmean_custom(var_subset)
        else:
            var_mean_values[i] = np.nan
        
    return var_mean_values


In [ ]:
def findTmValue(clockLs, tm, method='L', epsilon=1e-3):
    """
    在排序好的时间戳列表（clockLs）中找到指定时间间隔（tm）之前或之后的索引位置。
    
    参数:
        clockLs (np.array): 排序好的时间戳数组。
        tm (float): 需要查找的时间间隔，可以向前或向后。
        method (str): 查找方法（'F' 表示向未来查找，'L' 表示向过去查找）。
        epsilon (float): 用于调整精度问题的小数。
    
    返回:
        np.array: 对应于时间间隔 tm 之前或之后的索引数组。
                  当未来无有效时间戳时索引设为 -1，过去无有效时间戳时设为 0。
    """
    maxIx = len(clockLs)  # 获取时间列表的长度
    orignIx = np.arange(maxIx)  # 原始索引数组
    if method == 'F':
        # 向未来查找的情况
        ix = np.searchsorted(clockLs, clockLs + (tm - epsilon))  # 寻找每个时间点加上时间间隔后的位置
        # 创建一个掩码，标记边界条件：当前索引等于搜索到的索引减一、当前索引等于搜索到的索引，或搜索到的索引等于 clockLs 的长度
        mask = (orignIx == (ix - 1)) | (orignIx == ix) | (ix >= maxIx-1)
    elif method == 'L':
        # 向过去查找的情况
        ix = np.searchsorted(clockLs, clockLs - (tm - epsilon)) - 1  # 寻找每个时间点减去时间间隔后的位置
        ix = np.maximum(ix, 0)  # 确保索引不会小于0
        # 创建一个掩码，处理边界情况，包括当前索引等于搜索到的索引，或者计算后的时间小于 clockLs 的第一个元素
        mask = (orignIx == ix) | ((clockLs - (tm - epsilon)) < clockLs[0])

    ix[mask] = -1  
    return ix



In [ ]:
@jit(nopython=True)
def conti_price_chg(mid_price_values: np.ndarray) -> np.ndarray:
    last_mid = 0
    result = np.arange(len(mid_price_values))
    count = 0
    trend = 1
    value = 0
    for mid_price in mid_price_values:
        if mid_price > last_mid:
            this_change = 1
        elif mid_price < last_mid:
            this_change = -1
        else:
            this_change = 0
        if this_change == 0:
            value += 0

        elif this_change > 0:
            if trend == 0:
                value += 0
            elif trend > 0:
                value += 1
            else:
                value = 1
                trend = 1
        else:
            if trend == 0:
                value -= 0
            elif trend < 0:
                value -= 1
            else:
                value = -1
                trend = -1
        last_mid = mid_price
        result[count] = value
        count += 1

    return result[-1]

@jit(nopython=True)
def get_seconds_conti_price_chg(var_values, backward_idx_st, backward_idx_ed):
    res = np.full(len(var_values), np.nan)
    for i in prange(len(var_values)):
        start_idx = backward_idx_st[i]
        end_idx = backward_idx_ed[i]
        if i > start_idx and end_idx > start_idx:
            var_subset = var_values[start_idx:end_idx+1]
            res[i] = conti_price_chg(var_subset)
    return res


@jit(nopython=True, parallel=True)
def get_seconds_var_trend_st_ed(var_values, backward_idx_st, backward_idx_ed):
    var_pos_values = np.full(len(var_values), np.nan)
    var_neg_values = np.full(len(var_values), np.nan)
    for i in prange(len(var_values)):
        start_idx = backward_idx_st[i]
        end_idx = backward_idx_ed[i]
        if i > start_idx and end_idx > start_idx:
            var_subset = var_values[start_idx:end_idx+1]
            var_pos_values[i], var_neg_values[i] = nantrend_custom(var_subset)
        else:
            var_pos_values[i] = var_neg_values[i] = np.nan
        
    return var_pos_values, var_neg_values


In [140]:
path = '/root/TowerResearch'
df = pd.read_parquet(f"{path}/cu_new.parquet")

# Pick data in January
df = df.loc[df.index.month == 1]

# add RetMask 
df['RetMask'] = (df.index.second == 0) & (df.index.microsecond == 0)

# Removing the point with a mask of -1 is the last minute of a session
df = df[df['mask'] != -1]

cols = ['TradingDay','LastPrice', 'Volume', 'Turnover', 'OpenInterest', 'ClosePrice',
        'BidPrice1', 'BidVolume1', 'AskPrice1', 'AskVolume1',
       'BidPrice2', 'BidVolume2', 'AskPrice2', 'AskVolume2', 'BidPrice3',
       'BidVolume3', 'AskPrice3', 'AskVolume3', 'BidPrice4', 'BidVolume4',
       'AskPrice4', 'AskVolume4', 'BidPrice5', 'BidVolume5', 'AskPrice5',
       'AskVolume5', 'AveragePrice', 'delta_Volume', 'delta_Turnover',
       'Datetime', 'session', 'ActualTime',
       'intdate', 'clockLs', 'macro_price', 'forward_60s_idx', 'forward_60s_ret', 'RetMask']

df = df[cols]

# create time points
time_points = [1, 5, 10, 20, 40, 60]
for second in time_points:
    df[f'back_{second}s'] = findTmValue(df['clockLs'].values, tm=second, method='L')
df['back_0s'] = np.arange(df.shape[0])
df.loc[df[f'back_60s']==-1] = np.nan

# create basic datanodes
multiplier = 5
df['av_tt'] = np.nansum(df[[f"AskVolume{i}" for i in range(1, 6)]], axis=1)
df['bv_tt'] = np.nansum(df[[f"BidVolume{i}" for i in range(1, 6)]], axis=1)
df['ask_amt'] = np.nansum(df[[f"AskVolume{i}" for i in range(1,6)]].values * df[[f"AskPrice{i}" for i in range(1,6)]].values, axis=1)
df['bid_amt'] = np.nansum(df[[f"BidVolume{i}" for i in range(1,6)]].values * df[[f"BidPrice{i}" for i in range(1,6)]].values, axis=1)
df['ask_vwap'] = df['ask_amt'] / df['av_tt']
df['bid_vwap'] = df['bid_amt'] / df['bv_tt']
df['order_vwap'] = (df['ask_vwap'] + df['bid_vwap']) / 2
df['vwap'] = df['delta_Turnover'] / df['delta_Volume'] / multiplier
df['vwap_notnan'] = df['vwap'].fillna(method='ffill')
df['wp'] = (df['BidVolume1'] * df['BidPrice1'] + df['AskVolume1'] * df['AskPrice1']) / (df['BidVolume1'] + df['AskVolume1'])
df['mid'] = (df['AskPrice1'] + df['BidPrice1']) / 2

df.to_parquet('cu_1month.parquet')

# Construct Factors

In [2]:
df = pd.read_parquet('cu_1month.parquet')

In [3]:
df.columns

Index(['TradingDay', 'LastPrice', 'Volume', 'Turnover', 'OpenInterest',
       'ClosePrice', 'BidPrice1', 'BidVolume1', 'AskPrice1', 'AskVolume1',
       'BidPrice2', 'BidVolume2', 'AskPrice2', 'AskVolume2', 'BidPrice3',
       'BidVolume3', 'AskPrice3', 'AskVolume3', 'BidPrice4', 'BidVolume4',
       'AskPrice4', 'AskVolume4', 'BidPrice5', 'BidVolume5', 'AskPrice5',
       'AskVolume5', 'AveragePrice', 'delta_Volume', 'delta_Turnover',
       'Datetime', 'session', 'ActualTime', 'intdate', 'clockLs',
       'macro_price', 'forward_60s_idx', 'forward_60s_ret', 'RetMask',
       'back_1s', 'back_5s', 'back_10s', 'back_20s', 'back_40s', 'back_60s',
       'back_0s', 'av_tt', 'bv_tt', 'ask_amt', 'bid_amt', 'ask_vwap',
       'bid_vwap', 'order_vwap', 'vwap', 'vwap_notnan', 'wp', 'mid'],
      dtype='object')

In [5]:
time_points = [1, 5, 10, 20, 40, 60]

## Order type

In [12]:
df['A0'] = (df['AskVolume1'] - df['BidVolume1']) / (df['AskVolume1'] + df['BidVolume1'])
df['A1'] = get_seconds_var_mean_st_ed(df['A0'].values, df[f'back_1s'].values, np.maximum(0, df[f'back_0s'].values-1))
for i in range(1, len(time_points)):
    df[f"A{i+1}"] = get_seconds_var_mean_st_ed(df['A0'].values, df[f'back_{time_points[i]}s'].values, df[f'back_{time_points[i-1]}s'].values)

In [13]:
df[[f'A{i}' for i in range(len(time_points)+1)]].corr()

,A0,A1,A2,A3,A4,A5,A6
A0,1.000000,0.493633,0.368230,0.193227,0.129640,0.090158,0.064157
A1,0.493633,1.000000,0.562306,0.213520,0.134871,0.092321,0.066112
A2,0.368230,0.562306,1.000000,0.523130,0.252561,0.147800,0.097729
A3,0.193227,0.213520,0.523130,1.000000,0.460609,0.187990,0.111712
A4,0.129640,0.134871,0.252561,0.460609,1.000000,0.372106,0.152033
A5,0.090158,0.092321,0.147800,0.187990,0.372106,1.000000,0.345416
A6,0.064157,0.066112,0.097729,0.111712,0.152033,0.345416,1.000000


- There is basically no high correlation

In [14]:
df['B0'] = (df['av_tt'] - df['bv_tt']) / (df['av_tt'] + df['bv_tt'])
df['B1'] = df['B0'].diff()
for i in range(1, len(time_points)):
    df[f"B{i+1}"] = get_seconds_var_mean_st_ed(df['B1'].values, df[f'back_{time_points[i]}s'].values+1, df[f'back_{time_points[i-1]}s'].values)

In [15]:
df[[f'B{i}' for i in range(len(time_points)+1)]].corr()

,B0,B1,B2,B3,B4,B5,B6
B0,1.000000,0.153360,0.146969,0.104935,0.115996,0.120101,0.086678
B1,0.153360,1.000000,-0.089245,-0.018366,-0.012815,-0.006269,-0.002379
B2,0.146969,-0.089245,1.000000,-0.257539,-0.061731,-0.031297,-0.012185
B3,0.104935,-0.018366,-0.257539,1.000000,-0.255471,-0.044051,-0.018554
B4,0.115996,-0.012815,-0.061731,-0.255471,1.000000,-0.252329,-0.038140
B5,0.120101,-0.006269,-0.031297,-0.044051,-0.252329,1.000000,-0.269750
B6,0.086678,-0.002379,-0.012185,-0.018554,-0.038140,-0.269750,1.000000


- It can be seen that there is very low correlation between Bis

Haven't considered grouping session and Date yet
1. df.groupby('session')['order_imb1'].diff()
2. df.groupby('TradingDay')['order_imb1'].diff()

## Price type

In [ ]:
df['C0'] = df['mid'].pct_change()
df['C1'] = df['macro_price'].pct_change()
df['C2'] = df['wp'].pct_change()
df['C3'] = df['order_vwap'].pct_change()
df['C4'] = df['LastPrice'].pct_change()
df['C5'] = df['vwap_notnan'].pct_change()

df['C6'] = df['vwap_notnan'] / df['mid'] - df['wp'] / df['macro_price']

In [17]:
df[[f'C{i}' for i in range(7)]].corr()

,C0,C1,C2,C3,C4,C5,C6
C0,1.000000,0.739462,0.913069,0.909010,0.415505,0.382588,-0.306330
C1,0.739462,1.000000,0.400647,0.825084,0.422306,0.406025,0.008373
C2,0.913069,0.400647,1.000000,0.737363,0.309687,0.274749,-0.421992
C3,0.909010,0.825084,0.737363,1.000000,0.441201,0.422017,-0.187040
C4,0.415505,0.422306,0.309687,0.441201,1.000000,0.880959,0.100875
C5,0.382588,0.406025,0.274749,0.422017,0.880959,1.000000,0.145277
C6,-0.306330,0.008373,-0.421992,-0.187040,0.100875,0.145277,1.000000


- It can be seen that C0, C1, C2, C3 have high correlation, C4 and C5 have high correlation

In [18]:
df[f"C10"] = df[f"C1"]
for i in range(1, len(time_points)):
    df[f"C1{i}"] = get_seconds_var_mean_st_ed(df['C10'].values, df[f'back_{time_points[i]}s'].values+1, df[f'back_{time_points[i-1]}s'].values)

In [19]:
df[[f'C1{i}' for i in range(len(time_points))]].corr()

,C10,C11,C12,C13,C14,C15
C10,1.000000,0.000675,0.013091,0.007898,-0.000329,-0.000627
C11,0.000675,1.000000,0.012107,0.020355,0.004403,0.003443
C12,0.013091,0.012107,1.000000,0.020349,0.011088,0.002113
C13,0.007898,0.020355,0.020349,1.000000,0.024300,0.010357
C14,-0.000329,0.004403,0.011088,0.024300,1.000000,0.021796
C15,-0.000627,0.003443,0.002113,0.010357,0.021796,1.000000


In [20]:
df[f"C50"] = df[f"C5"]
for i in range(1, len(time_points)):
    df[f"C5{i}"] = get_seconds_var_mean_st_ed(df['C50'].values, df[f'back_{time_points[i]}s'].values+1, df[f'back_{time_points[i-1]}s'].values)

In [21]:
df[[f'C5{i}' for i in range(len(time_points))]].corr()

,C50,C51,C52,C53,C54,C55
C50,1.000000,-0.068009,0.002066,0.002787,0.000222,-0.001302
C51,-0.068009,1.000000,-0.102525,0.008043,0.001770,0.000297
C52,0.002066,-0.102525,1.000000,-0.051649,0.005477,-0.000374
C53,0.002787,0.008043,-0.051649,1.000000,-0.012658,0.006646
C54,0.000222,0.001770,0.005477,-0.012658,1.000000,-0.007913
C55,-0.001302,0.000297,-0.000374,0.006646,-0.007913,1.000000


In [22]:
df[f"C60"] = df[f"C6"]
for i in range(1, len(time_points)):
    df[f"C6{i}"] = get_seconds_var_mean_st_ed(df['C60'].values, df[f'back_{time_points[i]}s'].values, df[f'back_{time_points[i-1]}s'].values)

In [23]:
df[[f'C6{i}' for i in range(len(time_points))]].corr()

,C60,C61,C62,C63,C64,C65
C60,1.000000,0.276880,0.109834,0.069784,0.049016,0.034274
C61,0.276880,1.000000,0.425134,0.156553,0.085808,0.055032
C62,0.109834,0.425134,1.000000,0.356591,0.113411,0.063883
C63,0.069784,0.156553,0.356591,1.000000,0.273754,0.094337
C64,0.049016,0.085808,0.113411,0.273754,1.000000,0.249898
C65,0.034274,0.055032,0.063883,0.094337,0.249898,1.000000


In [35]:
df[[f'C6{i}' for i in range(len(time_points))]].isna().sum()

C60     120
C61    6183
C62    4691
C63    1847
C64    2541
C65    4882
dtype: int64

## Price Volume

In [26]:
df['D0'] = (df['wp'] - df['order_vwap']).rolling(60).sum() * df['delta_Volume'].rolling(60).sum() 
df['D1'] = ((df['wp'] - df['order_vwap']) * df['delta_Volume']).rolling(60).sum() / df['delta_Volume'].rolling(60).sum()

## Other type

In [31]:
df['Z1'] = (df['AskPrice1'] - df['BidPrice1']) / df['mid']
df['Z2'] = df['Z1'].diff()

In [55]:
pos, neg = get_seconds_var_trend_st_ed(df['macro_price'].diff().values, df['back_60s'].values, df['back_0s'].values)
df['Z3'] = (pos - neg) / (pos + neg + 1e-6)
df['Z4'] = utils.get_seconds_conti_price_chg(df['mid'].values, df['back_60s'].values, df['back_0s'].values)

In [ ]:
+1 - 1
+1 - 2
+1 - 3
0 - 0



# Test IC for factors

In [73]:
xVarsHash = [f'A{i}' for i in range(len(time_points)+1)] + \
            [f'B{i}' for i in range(len(time_points)+1)] + \
            [f'C1{i}' for i in range(len(time_points))] + \
            [f'C5{i}' for i in range(len(time_points))] + \
            [f'C6{i}' for i in range(len(time_points))] + \
            ['D0', 'D1'] + \
            ['Z1', 'Z2', 'Z3', 'Z4']

yVar = ['forward_60s_ret']

In [86]:
# xVarsHash = ['D0', 'D1']#[f'C1{i}' for i in range(len(time_points))]
xVarsSelected = []

data = df.loc[df.RetMask==True]
data = data[xVarsHash + yVar]

data = data.fillna(method='ffill')
Y = data.forward_60s_ret

for x in xVarsHash:
    if data[x].std() > 1e-6:
        xVarsSelected.append(x)

xVar = data[xVarsSelected]
common_notnan = xVar.notnull().all(axis=1) & Y.notnull()
data = data[common_notnan]
xVar_mean = xVar[common_notnan].mean()
xVar_std = xVar[common_notnan].std()
xVar_zscore = (xVar[common_notnan] - xVar_mean) / xVar_std
Y = Y[common_notnan]

In [93]:
pearson_corr = xVar_zscore.apply(lambda x: x.corr(Y, method='pearson'))
xVar_corr_matrix = xVar_zscore.corr()

# 按照绝对值高低排序输出X和Y之间的皮尔逊相关系数
pearson_corr_sorted = pearson_corr.abs().sort_values(ascending=False)
print(pearson_corr_sorted)

C14    0.059118
C54    0.055378
D1     0.052577
B2     0.050794
C50    0.049879
D0     0.049587
C62    0.049542
C15    0.045739
A3     0.040921
C55    0.040217
A4     0.036864
C61    0.035939
A0     0.033268
A2     0.032530
Z4     0.032422
C63    0.032278
Z3     0.032265
C10    0.031283
A1     0.028335
Z2     0.026535
C13    0.025227
C60    0.025183
B4     0.022472
A5     0.022352
C64    0.021181
A6     0.015686
B5     0.014935
C51    0.013282
B6     0.012591
B0     0.011484
B3     0.011356
C53    0.011057
Z1     0.010638
C11    0.010044
C12    0.005662
B1     0.005177
C52    0.003339
C65    0.002061
dtype: float64


In [88]:
from sklearn.linear_model import LassoLarsIC
# iss
lr_ = LassoLarsIC(fit_intercept=False)
lr_.fit(xVar_zscore, Y)
data['pred'] = lr_.predict(xVar_zscore.replace([np.inf, -np.inf, np.nan], 0))
pred_y = data[['pred', 'forward_60s_ret']].dropna()
train_corr = pearsonr(pred_y['pred'], pred_y['forward_60s_ret'])[0]
print(train_corr)

0.13290074011112643


# to be done

- break的gap应该去掉吗？
- factors distribution --- math transform
- case by case
    - price chg
    - orderimb chg
    - spread chg
    - volatility of vars above
- other time segment method?